# ICLR18repr demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using Hierarchichal Architecture search on MNIST

In [1]:
from math import pi, floor
import random
import hierarchy
import pickle
import numpy as np
import keras

Using TensorFlow backend.


### Sample problem: MNIST data set

The MNIST data set of handwritten gray scale images of digits 0-9 is a classic computer vision data set and therefore makes for good testing. Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
trainx = np.reshape(x_train,(60000,28,28,1))
testx = np.reshape(x_test, (10000,28,28,1))
trainy = keras.utils.np_utils.to_categorical(y_train, 10)
testy = keras.utils.np_utils.to_categorical(y_test, 10)
data = [trainx, trainy, testx, testy]

### Flat architectures
The simplest archecture described in the ICLR paper, technically a hierarchical architecture with layers = 2

In [3]:
def create_flat_population(pop_size, indiv_size):
    population = []
    for i in range(pop_size):
        population.append(hierarchy.FlatArch(indiv_size))
    return population

def assemble_small(architecture, inputdim):
    inputs = keras.layers.Input(shape=inputdim)
    l1 = keras.layers.Conv2D(16, kernel_size=3, activation='relu')(inputs)
    c1 = architecture.assemble(l1, 16)
    l2 = keras.layers.SeparableConv2D(32, kernel_size=3, strides=2, activation='relu')(c1)
    c2 = architecture.assemble(l2, 32)
    l3 = keras.layers.SeparableConv2D(64, kernel_size=3, strides=2, activation='relu')(c2)
    c3 = architecture.assemble(l3, 64)
    l4 = keras.layers.SeparableConv2D(64, kernel_size=3, strides=1, activation='relu')(c3)
    l5 = keras.layers.GlobalAveragePooling2D()(l4)
    outputs = keras.layers.Dense(10, activation='softmax')(l5)
    model = keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by the architecture on NMIST and then use their accuracy after 10 epochs as their fitness.

In [4]:
def fitness(architecture, data):
    network = assemble_small(architecture, data[0].shape[1:])
    network.fit(data[0], data[1],  epochs=5)
    loss, acc = network.evaluate(data[2], data[3])
    architecture.fitness = acc

### Random Search
The authors of this paper report even a simpler search than evolution, simply mutating randomly for a long period and then assessing fitness. This is implemented below.

In [5]:
def random_mutate(population, steps):
    for i in range(steps):
        indiv = random.choice(population)
        indiv.mutate()

In [7]:
pop = create_flat_population(25, 8)
random_mutate(pop, 100)
for i in range(len(pop)):
    print("Network " + str(i))
    fitness(pop[i], data)
pop.sort()
pop.reverse()
for indiv in pop:
    print(indiv.fitness)

Network 0
Epoch 1/5
15584/60000 [======>.......................] - ETA: 1:29 - loss: 1.6997 - acc: 0.4045

KeyboardInterrupt: 

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [10]:
def evolve(n, debugging=False):
    if(debugging):
        debug = open("debug.txt", "w")
    else:
        debug = None
    config.load('configMNIST')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(20, chromosome.ModuleChromo, debug=debug)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(5, chromosome.BlueprintChromo, module_pop, debug=debug)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 10, fitness, data, save_best=True, name='MNIST', debug=debug)
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="MNISTmod_")
    visualize.plot_stats(blueprint_pop.stats, name="MNISTbp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [6]:
evolve(5, True)

-----Generation 0--------
Network 0
Epoch 1/5
60000/60000 [==============================] - 4s 72us/step - loss: 1.9387 - acc: 0.8031
Epoch 2/5
60000/60000 [==============================] - 4s 70us/step - loss: 0.2887 - acc: 0.9327
Epoch 3/5
60000/60000 [==============================] - 4s 70us/step - loss: 0.2532 - acc: 0.9446
Epoch 4/5
60000/60000 [==============================] - 4s 71us/step - loss: 0.2529 - acc: 0.9505
Epoch 5/5
10000/10000 [==============================] - 0s 33us/step

Network 0 Fitness: 0.9407
Network 1
Epoch 1/5
60000/60000 [==============================] - 3s 51us/step - loss: 8.4975 - acc: 0.4642
Epoch 2/5
60000/60000 [==============================] - 3s 50us/step - loss: 4.5816 - acc: 0.7068
Epoch 3/5
60000/60000 [==============================] - 3s 52us/step - loss: 4.2154 - acc: 0.7326
Epoch 4/5
60000/60000 [==============================] - 3s 53us/step - loss: 4.0861 - acc: 0.7421
Epoch 5/5
10000/10000 [==============================] - 0s 24us/

60000/60000 [==============================] - 4s 66us/step - loss: 6.0411 - acc: 0.6228
Epoch 4/5
60000/60000 [==============================] - 4s 67us/step - loss: 5.9790 - acc: 0.6275
Epoch 5/5
10000/10000 [==============================] - 0s 45us/step

Network 2 Fitness: 0.6264
Network 3
Epoch 1/5
60000/60000 [==============================] - 9s 145us/step - loss: 6.9679 - acc: 0.5538
Epoch 2/5
60000/60000 [==============================] - 8s 139us/step - loss: 4.7874 - acc: 0.6971
Epoch 3/5
60000/60000 [==============================] - 9s 143us/step - loss: 4.3907 - acc: 0.7237
Epoch 4/5
60000/60000 [==============================] - 8s 141us/step - loss: 3.9939 - acc: 0.7494
Epoch 5/5
10000/10000 [==============================] - 1s 61us/step

Network 3 Fitness: 0.8005
Network 4
Epoch 1/5
60000/60000 [==============================] - 5s 75us/step - loss: 2.9195 - acc: 0.5445
Epoch 2/5
60000/60000 [==============================] - 4s 71us/step - loss: 1.0714 - acc: 0.7005


60000/60000 [==============================] - 7s 114us/step - loss: 1.1251 - acc: 0.6188
Epoch 3/5
60000/60000 [==============================] - 7s 119us/step - loss: 0.9514 - acc: 0.7024
Epoch 4/5
60000/60000 [==============================] - 7s 114us/step - loss: 0.9046 - acc: 0.7480
Epoch 5/5
10000/10000 [==============================] - 1s 92us/step

Network 7 Fitness: 0.9052
Network 8
Epoch 1/5
60000/60000 [==============================] - 5s 81us/step - loss: 0.7365 - acc: 0.8425
Epoch 2/5
60000/60000 [==============================] - 4s 63us/step - loss: 0.2890 - acc: 0.9304
Epoch 3/5
60000/60000 [==============================] - 4s 63us/step - loss: 0.2587 - acc: 0.9395
Epoch 4/5
60000/60000 [==============================] - 4s 63us/step - loss: 0.2469 - acc: 0.9456
Epoch 5/5
10000/10000 [==============================] - 1s 80us/step

Network 8 Fitness: 0.9498
Network 9
Epoch 1/5
60000/60000 [==============================] - 8s 131us/step - loss: 2.3020 - acc: 0.2948


In [7]:
def eval_best(model_file):
    config.load('configMNIST')
    model = keras.models.load_model(model_file)
    visualize.draw_net(model, "_" + model_file)    
    model.fit(data[0], data[1], epochs=5)
    loss, fitness = model.evaluate(data[2], data[3])
    print("fitness", fitness)

In [8]:
eval_best("MNIST_best_model_0")

Epoch 1/5
60000/60000 [==============================] - 6s 93us/step - loss: 0.2968 - acc: 0.9524
Epoch 2/5
60000/60000 [==============================] - 5s 89us/step - loss: 0.3103 - acc: 0.9497
Epoch 3/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.3061 - acc: 0.9537
Epoch 4/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.3506 - acc: 0.9510
Epoch 5/5
10000/10000 [==============================] - 1s 112us/step
fitness 0.9517
